In [1]:
from pdf_handler import PDFHandler

import fitz
import pandas as pd
import numpy as np
import os
import sqlite3


In [2]:
df = pd.read_csv("regex_validation.csv")
df

,pdf_name,section_type
0,amraoui2022_splc,numeric_section
1,cheng2023_tse,numeric_section
2,dorn2020_vamos,numeric_section
3,friesel2022_icse,numeric_section
4,gong2022_msr,numeric_section
...,...,...
124,tipu2022_cc,numeric_section
125,vázquez-ingelmo2020_cc,numeric_section
126,velez2020ase,numeric_section
127,Vitui2021ese,numeric_section


In [3]:
def find_all_paths_and_names(folder_path):
    all_paths = []
    all_names = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            all_paths.append(os.path.join(root, file))
            all_names.append(file)
    return all_paths, all_names

# Example usage
all_file_paths, all_file_names = find_all_paths_and_names("/home/pedro/Documents/Rag_test/grpc/papers_pdf")

In [4]:

#path = "/home/pedro/Documents/Rag_test/grpc/papers_pdf/ScienceDirect/Arcaini2020.pdf"
    
def extract_titles_sections(path, section_pattern):
    doc = PDFHandler.try_open(path)
    conn = sqlite3.connect("extracted_titles.db")
    cursor = conn.cursor()
    
    # Check if PDF already exists
    pdf_name = os.path.basename(path)
    cursor.execute("SELECT id FROM pdfs WHERE pdf_name = ?", (pdf_name,))
    existing_pdf = cursor.fetchone()
        
    if not existing_pdf:
        cursor.execute("INSERT INTO pdfs (pdf_name) VALUES (?)", (pdf_name,))
        pdf_id = cursor.lastrowid
    else:
        return -1
    
    # Extract text
    doc = PDFHandler.try_open(path)
    if doc is None:
        print("Error opening the PDF")
        exit(1)

    text, page_count = PDFHandler.simple_extraction(doc)
    #tagged_text, tagged_page_count = PDFHandler.tagged_text_extraction(doc)
    # print(f"Tagged text length: {tagged_text}")
    # Extract all sections
    sections = PDFHandler.extract_all_pdf_sections(text, section_pattern, page_count)
    
    # Insert extracted text into the table with sequence
    sequence = 1
    for section in sections:
        cursor.execute("""
            INSERT INTO extracted_text (pdf_id, sequence, title,confidence_score, text) 
            VALUES (?, ?, ?, ?, ?)
        """, (pdf_id, sequence, section.section_title, section.confidence_score, section.content))
        sequence += 1
    
    # Commit the changes and close the connection
    conn.commit()
    conn.close()
    doc.close()
    
    return pdf_id

In [5]:
# Change the path to your folder with PDF files
all_file_paths, all_file_names = find_all_paths_and_names("/home/PUC/Documentos/AutoSLR/papers_pdf")

In [6]:
conn = sqlite3.connect("extracted_titles.db")
cursor = conn.cursor()

# Create tables if they don't exist
cursor.execute("""
    CREATE TABLE IF NOT EXISTS pdfs (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        pdf_name TEXT NOT NULL UNIQUE,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
""")

cursor.execute("""
    CREATE TABLE IF NOT EXISTS extracted_text (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        pdf_id INTEGER NOT NULL,
        sequence INTEGER NOT NULL,
        title TEXT,
        text TEXT,
        confidence_score INTEGER,
        FOREIGN KEY (pdf_id) REFERENCES pdfs (id)
    )
""")

In [7]:
for idx, file in enumerate(all_file_paths):
    try:
        section_pattern = df.loc[df["pdf_name"] == all_file_names[idx][:-4], "section_type"].values[0]
        if section_pattern == "null":
            continue
        extract_titles_sections(file, section_pattern)
    except Exception as e:
        print(f"Error processing {file}: {e}")
        continue

INTRODUCTION 1
MOTIVATION 2
DEFINITIONS AND PROBLEM STATEMENT 3
BEETLE: BELLWETHER TRANSFER LEARNER 4
OTHER TRANSFER LEARNING METHODS 5
EXPERIMENTAL SETUP 6
RESULTS 7
DISCUSSION 8
RELATED WORK 10
CONCLUSION 11
Abstract 2022
INSA Rennes 3
Introduction 1
Background and motivation 2
Research questions 3
Experimental Protocol 4
Based on our last check on February 2022 9
Results 5
Discussion 6
Threats to validity 7
Related Work 8
Conclusion 9
INTRODUCTION I
PRELIMINARIES II
EXPERIMENT SETUP IV
RESULTS V
CONCLUSION VI
ACKNOWLEDGMENTS VII
INTRODUCTION 1
Exploration: new commits 0
AN ALGORITHM FOR CHANGE POINT 3
Time 1000
EVALUATION 4
LRZIP 16
OGGENC 9
Iterations Required for Termination 20
DISCUSSION 5
RELATED WORK 6
CONCLUSION 7
ACKNOWLEDGMENTS 8
Introduction 1
Background 2
Study settings 3
Results 4
Threats to validity 5
Conclusion 6
Introduction 1
Background 2
Related work 3
The proposed approach for locating features implementations 4
Results and discussion 5
Conclusion and future work 6
